In [126]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [127]:
geolocator = Nominatim(user_agent="geoapiExercises")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [128]:
columns = ['datetime', 'city', 'state', 'country', 'shape', 'duration_s', 'hours', 'comments', 'date_p', 'latitude', 'longitude']
df = pd.read_csv("completo.csv", header=0, names=columns, error_bad_lines=False, warn_bad_lines=False, low_memory=False)
df.head()

,datetime,city,state,country,shape,duration_s,hours,comments,date_p,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [129]:
df.shape

(88679, 11)

In [130]:
#dropando colunas que não serão utilizadas

df = df[df['state'].isna()]

In [131]:
df.shape

(7409, 11)

In [132]:
df['latitude'] = df['latitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['longitude'] = df['longitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [133]:
df_lat_lon = df[['latitude', 'longitude']]
df_lat_lon.head(12)

,latitude,longitude
2,53.200000,-2.916667
6,51.434722,-3.180000
18,32.364167,-64.678611
21,51.500000,-3.200000
25,51.783333,-0.783333
36,-38.662334,178.017649
40,53.800000,-1.583333
58,55.183333,59.650000
69,-33.093373,151.588982
76,0.000000,0.000000


In [134]:
for index, row in df_lat_lon.iterrows():
    df_lat_lon.at[index, 'location'] = str(row[0]) + ', ' + str(row[1])

df_lat_lon.drop('latitude', axis=1, inplace=True)
df_lat_lon.drop('longitude', axis=1, inplace=True)

df_lat_lon

C:\Users\rober\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\rober\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\rober\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

,location
2,"53.2, -2.916667"
6,"51.434722, -3.18"
18,"32.364167, -64.678611"
21,"51.5, -3.2"
25,"51.783333, -0.783333"
...,...
88570,"57.166667000000004, -2.666667"
88590,"53.166667000000004, -1.0"
88591,"49.45, 7.75"
88623,"0.0, 0.0"


In [135]:
df_lat_lon.to_csv('lat_lon.csv')

In [136]:
chunker = pd.read_csv("lat_lon.csv", header=0, error_bad_lines=False, warn_bad_lines=False, low_memory=False, chunksize=250)


In [ ]:
for piece in chunker:
    
    address = [reverse(loc) for loc in piece['location']]
    
    centers_states = []
    for i in range(len(address)):
        try:
            centers_states.append(address[i].raw['address']['state'])
        except:
            centers_states.append('NULL')
    
    piece['state'] = centers_states
    piece.to_csv('pieces2.csv', mode = 'a', header = False)

In [98]:
for piece in chunker:
    
    piece['address'] = piece['location'].apply(lambda loc: reverse(loc) if loc else None)
    piece.to_csv('pieces.csv', mode = 'a', header = False)

In [122]:
df_pieces = pd.read_csv("pieces.csv", header=None)
df_pieces.drop(0, axis=1, inplace=True)
df_pieces.drop(1, axis=1, inplace=True)
df_pieces.rename(columns={2:'Lat_Lon', 3:'Address'}, inplace=True)
df_pieces

,Lat_Lon,Address
0,"53.2, -2.916667","Greyhound Retail Park, City of Chester, Cheste..."
1,"51.434722, -3.18","Victoria Square, Penarth Marina, Penarth, Vale..."
2,"32.364167, -64.678611","South Perimeter Road, Saint George's, DD03, Be..."
3,"51.5, -3.2",Cardiff University - Talybont North (Halls of ...
4,"51.783333, -0.783333","Stoke Mandeville Station, Meadow Park, Stoke M..."
...,...,...
7404,"57.166667000000004, -2.666667","Aberdeenshire, Scotland, United Kingdom"
7405,"53.166667000000004, -1.0","Red Hill Lane, Rufford CP, Eakring, Newark and..."
7406,"49.45, 7.75","Neumühle, Schulzentrum Nord, Engelshof, Erzhüt..."
7407,"0.0, 0.0",Soul Buoy


In [125]:
type(df_pieces['Address'][1])

str